In [1]:
import sys
import os
import shutil
import traceback

sys.path.append(os.path.abspath(os.getcwd()))
import Utils

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline


In [2]:

# Example
directory = Utils.Directory(case_name="test_case_7")
case_number = 0

# RVE Analysis
from RVE import RVE
panel = RVE(
    directory=directory,
    case_number=case_number,
)
panel.analysis()

	UPDATE: starting RVE analysis test_case_7 - 0
	UPDATE: analysis for test_case_7 - 0 completed in 21.48s


In [3]:
# Fairing Analysis
from Fairing import FairingAnalysis, FairingGeometry, FairingData
fairing = FairingAnalysis(
    RVE_identifier=panel.case_number, 
    directory=directory, 
    case_number=case_number,
)
fairing.analysis()



	UPDATE: starting FairingAnalysis analysis test_case_7 - 0
	UPDATE: analysis for test_case_7 - 0 completed in 15.60s


In [4]:
# Trace principal strain directions  
from Fairing import FairingData
from Tailoring import Lattice
lattice_data = Lattice(directory, case_number)
lattice_data.analysis()

Lattice field for folding angles (deg): ['0.0', '5.0', '10.0', '15.0']
	UPDATE: trace_streamlines for test_case_7 - 0 completed in 4.11s
	UPDATE: analysis for test_case_7 - 0 completed in 9.67s


In [21]:
# Generate field-aligned lattice
from Tailoring import Tailored    
tailored = Tailored(directory, case_number, lattice_data=lattice_data)
tailored.create_fairing_2D(bool_plot=True)

# 2D Mesh Plot
increment_key = "15"
# tailored.mesh2D[increment_key].plot_2D(
#     {"group_sets": [], "triangle_sets": []},
#     False,
#     os.path.join(directory.case_folder, "fig", f"{case_number}_initial_2D_{increment_key}.svg"),
#     True
# )


	UPDATE: create_fairing_2D for test_case_7 - 0 completed in 26.09s


In [22]:
# Mapping lattice to 3D wing
tailored.mapping_2D_to_3D(bool_plot=True)
tailored.validate_midplane_mesh()

# # 3D Mesh Plot
# increment_key = "15"
# tailored.mesh3D[increment_key].plot_3D(
#     {"beam_sets": [], "triangle_sets": []},
#     False,
#     os.path.join(directory.case_folder, "fig", f"{case_number}_tailored_3D_{increment_key}.svg"),
#     True
# )

In [ ]:

from scipy import interpolate

# data for interpolation
coords = fairing.aerofoil_coords['mid']
normals = Utils.GeoOps.normals_2D(coords)

# RBF interpolation
f_normals = interpolate.RBFInterpolator(coords, normals, neighbors=4, kernel='linear')

# # sampling
# inc = 4
# X=coords[::inc,0]
# Y=coords[::inc,1]
# N=f_normals(coords[::inc,:])
# plt.figure(figsize=(10,5))
# plt.plot(X, Y, 'r.-')
# plt.quiver(X, Y, N[:,0], N[:,1])
# plt.axis('equal')

# seperate
midplane = tailored.mesh3D[increment_key].copy()
inner = tailored.mesh3D[increment_key].copy()
outer = tailored.mesh3D[increment_key].copy()

# offset
inner.nodes[:,[0,2]] += f_normals(inner.nodes[:,[0,2]]) * fairing.RVE_variables["lz"]
outer.nodes[:,[0,2]] -= f_normals(outer.nodes[:,[0,2]]) * fairing.RVE_variables["lz"]

#



# # plot 3D mesh
# tailored.mesh3D[increment_key].plot_3D(
#     {"beam_sets": []},
#     False,
#     os.path.join(directory.case_folder, "fig", f"{case_number}_tailored_3D_{increment_key}.svg"),
#     True
# )

In [ ]:
# update nodes
nodes = original.nodes
node_offset = nodes.shape[0]
new_nodes = nodes.copy()
new_nodes[:,[0,2]] += f_normals(nodes[:,[0,2]]) * fairing.RVE_variables["lz"]
tailored.mesh3D[increment_key].nodes = np.vstack((nodes, new_nodes))
# update beam 
tailored.mesh3D[increment_key].beams = np.vstack((tailored.mesh3D[increment_key].beams, tailored.mesh3D[increment_key].beams + node_offset))
new_sets = {}
for key, value in tailored.mesh3D[increment_key].beam_sets.items():
    new_sets[f"{key}_inner"] = value + node_offset
tailored.mesh3D[increment_key].beam_sets.update(new_sets)
# update triangle 
tailored.mesh3D[increment_key].triangles = np.vstack((tailored.mesh3D[increment_key].triangles, tailored.mesh3D[increment_key].triangles + node_offset))
new_sets = {}
for key, value in tailored.mesh3D[increment_key].triangle_sets.items():
    new_sets[f"{key}_inner"] = value + node_offset
tailored.mesh3D[increment_key].triangle_sets.update(new_sets)

In [20]:


def write_mesh(self, filename):
    """
    Writes the mesh to a file using the specified serialization method.

    Parameters:
        filename (str): The base file path (without extension) where the mesh will be saved.
    """
    def format_lines(data_list, items_per_line=16):
        lines : list[str] = []
        for i in range(0, len(data_list), items_per_line):
            chunk = data_list[i:i + items_per_line]
            lines.append(", ".join(map(str, chunk)) + ("," if i + items_per_line < len(data_list) else ""))
        return lines

    lines = []

    # nodes
    lines.append("*NODE")
    for i, node in enumerate(self.nodes):
        i = i + 1  # Abaqus uses 1-based indexing
        lines.append("%i, %f, %f, %f" % (i, *node))

    # beams
    lines.append("*ELEMENT, TYPE=%s" % ("B31"))
    for i, beam in enumerate(self.beams):
        i = i + 1  # Abaqus uses 1-based indexing
        beam = beam + 1  # Abaqus uses 1-based indexing
        lines.append("%i, %i, %i" % (i, beam[0], beam[1]))

    # triangles
    num_beam_elements = self.beams.shape[0]
    lines.append("*ELEMENT, TYPE=%s" % ("S3"))
    # outside
    for i, triangle in enumerate(self.triangles):
        i = i + 1 + num_beam_elements # Abaqus uses 1-based indexing
        triangle = triangle + 1  # Abaqus uses 1-based indexing
        lines.append("%i, %i, %i, %i" % (i, triangle[0], triangle[1], triangle[2]))

    # node sets
    for set_name, set_items in self.nsets.items():
        lines.append("*NSET, NSET=%s" % (set_name))
        set_items = set_items + 1  # Abaqus uses 1-based indexing
        lines.extend(format_lines(set_items))
    # beam sets
    for set_name, set_items in self.beam_sets.items():
        set_items = set_items + 1  # Abaqus uses 1-based indexing
        lines.append("*ELSET, ELSET=%s" % (set_name))
        lines.extend(format_lines(set_items))
    # triangle sets
    for set_name, set_items in self.triangle_sets.items():
        set_items = set_items + 1  # Abaqus uses 1-based indexing
        lines.append("*ELSET, ELSET=%s" % (set_name))
        lines.extend(format_lines(set_items))

    # write to file
    with open(filename+".inp", "w", encoding="utf-8") as f:
        f.write("\n".join(lines) + "\n")


# write mesh
write_mesh(
    tailored.mesh3D[increment_key], 
    os.path.join(directory.case_folder, "mesh", f"{case_number}_tailored_3D_{increment_key}.inp")
)